The following notebook will explore any trends between the air quality data and the weather data. Both datasets contain hourly measurements.

In [38]:
# Import 3rd party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')


In [46]:
# Switch to data cleaning branch and read in cleaned air quality and weather dataframes
#!git   checkout 1-Data_cleaning
weather_data = pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\1-Data_cleaning\weather_data_cleaned.csv")
aq_data = pd.read_csv(r"C:\Users\user\PycharmProjects\CIV1498Project2\1-Data_cleaning\aq_data_cleaned.csv")

In [50]:
# merge the two datasets based on their datetime indexes
aq_weather = weather_data.join(aq_data)

# view infromation and dataframe
print(aq_weather.info())
aq_weather.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Datetime         26304 non-null  object 
 1   TEMP_MEAN_(C)    26304 non-null  float64
 2   WDIR_VECT_(DEG)  26304 non-null  float64
 3   WDIR_UVEC_(DEG)  26304 non-null  float64
 4   WSPD_SCLR_(M/S)  26304 non-null  float64
 5   HUMIDITY_(%)     26304 non-null  float64
 6   Unnamed: 0       26304 non-null  int64  
 7   Date             26304 non-null  object 
 8   Time             26304 non-null  object 
 9   NOx              24927 non-null  float64
 10  SO2              24791 non-null  float64
 11  NO               24927 non-null  float64
 12  NO2              24927 non-null  float64
 13  TRS              24567 non-null  float64
 14  O3               24944 non-null  float64
dtypes: float64(11), int64(1), object(3)
memory usage: 3.0+ MB
None


,Datetime,TEMP_MEAN_(C),WDIR_VECT_(DEG),WDIR_UVEC_(DEG),WSPD_SCLR_(M/S),HUMIDITY_(%),Unnamed: 0,Date,Time,NOx,SO2,NO,NO2,TRS,O3
0,2019-01-01 01:00:00,0.8,272.7,273.2,10.0,67.1,0,12/31/2018,24:00 AM,0.6,0.4,0.0,0.6,0.6,38.3
1,2019-01-01 02:00:00,0.8,274.6,275.0,8.8,66.8,1,1/1/2019,1:00 AM,1.2,0.4,0.2,1.0,0.7,37.6
2,2019-01-01 03:00:00,1.2,274.9,274.8,8.4,66.0,2,1/1/2019,2:00 AM,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 04:00:00,0.7,268.3,268.6,7.7,68.6,3,1/1/2019,3:00 AM,0.6,0.2,0.1,0.5,0.1,38.1
4,2019-01-01 05:00:00,1.1,263.5,263.6,10.4,69.6,4,1/1/2019,4:00 AM,1.7,0.1,0.1,1.6,0.0,36.7
